In [ ]:
# Load in software packages
%matplotlib inline
from parcels import FieldSet, Field, VectorField, ParticleFile, ParticleSet, JITParticle, ScipyParticle, AdvectionRK4, AdvectionEE, plotTrajectoriesFile, Variable
from parcels import ErrorCode
from parcels import rng as random
import numpy as np
import math
from datetime import timedelta, datetime
from operator import attrgetter

In [ ]:
# Load in ocean model 
filenames = {'U':"chapter_4_models/copernicus_phy001_030/global-reanalysis-phy-001-030-daily_*.nc",
             'V':"chapter_4_models/copernicus_phy001_030/global-reanalysis-phy-001-030-daily_*.nc",
             'bathy':'chapter_4_models/copernicus_phy001_030_Statics/GLO-MFC_001_030_mask_bathy.nc',
             'mask':'chapter_4_models/copernicus_phy001_030_Statics/GLO-MFC_001_030_mask_bathy.nc',
             'S':"chapter_4_models/copernicus_phy001_030/global-reanalysis-phy-001-030-daily_*.nc"}
variables = {'U':'uo',
             'V':'vo',
             'S':'so',
             'mask':'mask',
            'bathy':'deptho'}
dimensions = {'U': {'lon': 'longitude', 'lat': 'latitude', 'time': 'time'},
              'V': {'lon': 'longitude', 'lat': 'latitude','time': 'time'},
              'S': {'lon': 'longitude', 'lat': 'latitude','time': 'time'},
              'mask': {'lon': 'longitude', 'lat': 'latitude', 'depth':'depth'},
              'bathy': {'lon': 'longitude', 'lat': 'latitude'}}

fieldsetcurrents = FieldSet.from_netcdf(filenames, variables, dimensions, allow_time_extrapolation=True)

In [ ]:
# Load in wind data

windfiles = {'U': "CERSAT/CERSAT-GLO-BLENDED_WIND_L4-V6-OBS_FULL_TIME_SERIE_1603883929498.nc",
             'V': "CERSAT/CERSAT-GLO-BLENDED_WIND_L4-V6-OBS_FULL_TIME_SERIE_1603883929498.nc"}
windvariables = {'U': 'eastward_wind',
                 'V': 'northward_wind'}
winddimensions = {'lat': 'lat',
                  'lon': 'lon',
                  'time': 'time'}

fieldsetwind = FieldSet.from_netcdf(windfiles, windvariables, winddimensions, allow_time_extrapolation=True)
#fieldsetwind.U.set_scaling_factor(0.01) # 1% of wind speed 
#fieldsetwind.V.set_scaling_factor(0.01) # 1% of wind speed 

In [ ]:
fieldset = FieldSet(U = fieldsetcurrents.U + fieldsetwind.U, V = fieldsetcurrents.V + fieldsetwind.V)

In [ ]:
fieldsetstatic = FieldSet.from_netcdf(filenames, variables, dimensions, allow_time_extrapolation=True)
fieldset.add_field(fieldsetstatic.mask)
fieldset.add_field(fieldsetstatic.S)

In [ ]:
class KelpParticle(ScipyParticle):  # Define a new particle class that contains three extra variables
    age = Variable('age', dtype=np.float32, initial=0.) # initialise age
    distance = Variable('distance', initial=0., dtype=np.float32)  # the distance travelled
    prev_lon = Variable('prev_lon', dtype=np.float32, to_write=False,
                        initial=attrgetter('lon'))  # the previous longitude
    prev_lat = Variable('prev_lat', dtype=np.float32, to_write=False,
                        initial=attrgetter('lat'))  # the previous latitude.
    VelU = Variable('VelU', dtype=np.float32) # create variable for meridonal velocity
    VelV = Variable('VelV', dtype=np.float32) # create variable for zonal velocity
    WindU = Variable('WindU')
    WindV = Variable('WindV')
    StokesU = Variable('StokesU')
    StokesV = Variable('StokesV')
    KelpVelU = Variable('KelpVelU')
    KelpVelV = Variable('KelpVelV')
    Phase = Variable('Phase', dtype=np.float32, initial = 1) # 1-floating, 2-beached, 3-out of bounds
    waterdragU = Variable('waterdragU')
    waterdragV = Variable('waterdragV')
    winddragU = Variable('winddragU')
    winddragV = Variable('winddragV')
    stokesdragU = Variable('stokesdragU')
    stokesdragV = Variable('stokesdragV')
    Salinity = Variable('Salinity', initial = fieldset.S, dtype=np.float32) # create variable for salinity
    Mask = Variable('Mask', initial = fieldset.mask, to_write = False) # create variable for the 


def SampleDistance(particle, fieldset, time): # Function measuring distance
    # Calculate the distance in latitudinal direction (using 1.11e2 kilometer per degree latitude)
    lat_dist = (particle.lat - particle.prev_lat) * 1.11e2
    # Calculate the distance in longitudinal direction, using cosine(latitude) - spherical earth
    lon_dist = (particle.lon - particle.prev_lon) * 1.11e2 * math.cos(particle.lat * math.pi / 180)
    # Calculate the total Euclidean distance travelled by the particle
    particle.distance += math.sqrt(math.pow(lon_dist, 2) + math.pow(lat_dist, 2))

    particle.prev_lon = particle.lon  # Set the stored values for next iteration.
    particle.prev_lat = particle.lat

# Hydrodynamic drag kernel
def WaterWindDrag(particle, fieldset, time):
        particle.VelU = fieldset.U[0][time, particle.depth, particle.lat, particle.lon]
        particle.VelV = fieldset.V[0][time, particle.depth, particle.lat, particle.lon]
        particle.Salinity = fieldset.S[time, particle.depth, particle.lat, particle.lon] # partical salinity units
        so = ((particle.Salinity/1000)*0.001) # conversion of salinity to correct units
        Ac =  310.54  # cross-sectional area of plant, must be entered manually and calculated before hand
        mass = 48.65  # total mass of the kelp plant being simulated
        expo = 0.85
        particle.waterdragU = ((0.5*so*(particle.VelU*particle.VelU)*0.47*Ac)*expo)/mass # syntax for calculating the loss in velocity as a result of drag
        particle.waterdragV = ((0.5*so*(particle.VelV*particle.VelV)*0.47*Ac)*expo)/mass
        
        
        particle.WindU = fieldset.U[1][time, particle.depth, particle.lat, particle.lon]
        particle.WindV = fieldset.V[1][time, particle.depth, particle.lat, particle.lon]
        AcW =  310.54 # cross-sectional area of plant, must be entered manually and calculated before hand
        massW = 48.65  # total mass of the kelp plant being simulated
        expoW = 0.15
        particle.winddragU = ((0.5*1.225*(particle.WindU*particle.WindU)*0.47*AcW)*expoW)/massW # syntax for calculating the loss in velocity as a result of drag
        particle.winddragV = ((0.5*1.225*(particle.WindV*particle.WindV)*0.47*AcW)*expoW)/massW
        particle.KelpVelU = (particle.VelU - (particle.waterdragU + particle.winddragU))
        particle.KelpVelV = (particle.VelV - (particle.waterdragV + particle.winddragV))
        particle.lon  += particle.KelpVelU * particle.dt   # overall velocity minus the loss in the velocity as a result of drag
        particle.lat  += particle.KelpVelV * particle.dt   # overall velocity minus the loss in the velocity as a result of drag
        
def Age(particle, fieldset, time):
        particle.age += (particle.dt/3600)
        
def Beaching(particle, fieldset, time):
        particle.Mask = fieldset.mask[time, particle.depth,particle.lat, particle.lon]
        if particle.Mask <= 0.3:
            particle.Phase = 2
            particle.delete()
            
def DeleteParticle(particle, fieldset, time): # Function that deletes particle if it goes out of bounds to avoid OutOfBounds error
    particle.delete()
    particle.Phase = 2       

In [ ]:
# Year1 Summer: 2017, 12, 1
# Year1 Autumn: 2018, 3, 1
# Year1 Winter: 2018, 6, 1
# Year1 Spring: 2018, 9, 1

pset = ParticleSet.from_list(fieldset=fieldset,   # the fields on which the particles are advected
                             pclass=KelpParticle,  # the type of particles (JITParticle/ScipyParticle/Custom)
                             lon=[18.3176], 
                             lat=[-34.1501],
                             repeatdt=None,        #43200(12hrs) # 21600 (6hrs)
                             time=(datetime(2018, 6, 1)))  

In [ ]:
## cast kernels, select appropriate 'kernel set'

kernels = pset.Kernel(SampleDistance) + WaterWindDrag + Age + Beaching # KelpFloat

In [ ]:
## execute simulation
## define output file

pset.execute(kernels, 
             runtime=timedelta(days=10),
             dt=timedelta(hours=3),
             output_file=pset.ParticleFile(name="test_landmask.nc", outputdt=timedelta(hours=3)), 
             recovery = {ErrorCode.ErrorOutOfBounds: DeleteParticle})

In [ ]:
# simple plot for viewing, use for checking simulation output

plotTrajectoriesFile('test_landmask.nc')
#plotTrajectoriesFile('test_B.nc')

In [ ]:
plotTrajectoriesFile('test_landmask.nc', mode='movie2d_notebook')  

In [ ]:
plotTrajectoriesFile('test_B.nc', mode='movie2d_notebook')  

In [ ]:
pset.show(field='vector', show_time=datetime(2018, 1, 30, 0), domain={'N':-30, 'S':-35, 'E':21, 'W':15}, savefile = 'vector_day30', with_particles = False, vmax=0.8)